# Assignment HPDM172

## Intro

This notebook documents the design and build of a relational database modeling the activties of a group of hospitals. The aim is to create a structured dataset that captures the key entities and interactions found in a healthcare environment which includes hospitals, doctors, patients, appointments, prescriptions, diseases, and laboratory investigations.

The database is implemented using MySQL, a relational database engine that integrates cleanly with Python. All tables, relationships, and data-generation steps are documented.


## Aims

1. Design a relational schema for modeling the activties of a group of hospitals
2. Implement the tables using SQL CREATE TABLE statements in Python
3. Generate synthetic data to populate the database
4. Demonstrate key database operations such as inserts and queries 

## Scope

The database includes tables representing:

- Hospitals 
- Doctors – clinical staff working at a hospital
- Patients – individuals receiving care, each assigned to one doctor
- Appointments – events where a doctor sees a patient at a hospital
- Medications – a catalogue of drug names
- Prescriptions – many-to-many link between patients and medications, with doctor + date information
- Diseases – conditions treated by doctors and medications
- DiseaseSpecialist – which doctors specialise in which diseases
- DiseaseTreatment – which medications treat which diseases
- LabTest – a catalogue of test types 
- LabResult – individual patient test results ordered by a doctor

## Structure of notebook(s)

1. Introduction
2. Database Initialisation
3. Schema Definition: For each table, a markdown explanation and then SQL CREATE TABLE code executed using a helper function
4. Sytntheic Data Generation
5. Data Validation and testing
6. Queries
7. Entity-Relationship Diagram

## Database Initialisation

This section sets up the MySQL database that will be used throughout the notebook.
Unlike SQLite, which is file-based, MySQL is a client–server relational database, so the workflow involves connecting to a running MySQL server, creating the database if it does not already exist, and then selecting it for use.

The following initialisation steps are performed:

Import the required Python library (mysql.connector)
This provides the Python interface for connecting to, creating, and querying a MySQL database.

Establish a connection to the MySQL server
Using a username, password, host, and (optionally) database name.
If the database does not yet exist, the notebook will create it.

Create the project database (hospitaldb) if necessary
This ensures the notebook is fully reproducible and can be run on any machine with MySQL installed.

Select the database and create a reusable cursor object
The cursor will be used for executing SQL commands throughout the notebook, including table creation, data insertion, and queries.

Once this initialisation step is complete, the conn (connection) and cur (cursor) objects will be available for all SQL operations in later sections of the notebook. This provides a consistent starting point for defining the schema, populating tables with synthetic data, and running analytical queries.

In [ ]:
"""
Creates the MySQL database (`hospitaldb`) if it does not already exist.

This initialisation step connects to the MySQL server using the supplied
credentials and executes a `CREATE DATABASE` statement. Because MySQL
operates as a client–server system rather than a file-based engine,
the database must be explicitly created before any tables can be defined.

Steps performed:
1. Connect to the MySQL server (without selecting a database).
2. Issue a `CREATE DATABASE IF NOT EXISTS` command to ensure the
   `hospitaldb` database is available for use.
3. Close the cursor and connection, as this step only handles
   database creation.

A separate connection—targeting the `hospitaldb` database—will be created
in the next step for running table creation statements and all subsequent
SQL operations.
"""


import mysql.connector

conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password=""
)

cur = conn.cursor()
cur.execute("CREATE DATABASE IF NOT EXISTS hospitaldb;")
cur.close()
conn.close()


In [4]:
"""
Initialises the MySQL database connection for the Hospital Information System.

This cell:
- Imports the required Python modules for MySQL connectivity
- Connects to a running MySQL server using user credentials
- Selects (or creates) the target database
- Creates a reusable cursor object for executing SQL statements
- Prints confirmation messages including the MySQL server version

In contrast to SQLite:
- MySQL uses a client–server architecture rather than a file-based database
- Foreign key constraints are always enforced when using the InnoDB engine
- No PRAGMA commands are needed
- Data types and AUTO_INCREMENT behaviour follow MySQL conventions

All subsequent CREATE TABLE and data insertion cells will rely on the
`conn` (connection) and `cur` (cursor) objects defined here.
"""

import mysql.connector
import pandas as pd

# --- Database connection settings (modify as needed) ---
db_config = {
    "host": "localhost",
    "user": "root",
    "password": "",
    "database": "hospitaldb"
}

# Establish a connection to the MySQL server and database.
conn = mysql.connector.connect(**db_config)

# Create a cursor object to execute SQL commands.
cur = conn.cursor()

# Show connection confirmation and server version.
cur.execute("SELECT VERSION();")
mysql_version = cur.fetchone()[0]

print("Connected to MySQL database:", db_config["database"])
print("MySQL server version:", mysql_version)



Connected to MySQL database: hospitaldb
MySQL server version: 9.5.0


## HOSPITAL table

This is the core table. Each hospital should have a unique name and a unique id.
There must be attributes like address, size etc

| Field Name             | Data Type       | Constraints & Notes                            |
| ---------------------- | --------------- | ---------------------------------------------- |
| `hospital_id`          | INT             | **Primary key**, auto-increment                |
| `name`                 | VARCHAR(150)    | **Unique**, not null                           |
| `address`              | VARCHAR(255)    | Not null                                       |
| `size_beds`            | INT             | Not null, must be >0                           |
| `accreditation_status` | VARCHAR(50)     | e.g. "Accredited", "Pending", "Not accredited" |


In [5]:
"""
Creates the `Hospital` table in the MySQL database.

This table stores core information about each hospital, including:
- A unique primary key (`hospital_id`)
- A unique hospital name
- Address information
- Bed capacity (`size_beds`)
- Type of hospital (e.g., General, Teaching, Specialist)
- Accreditation status

This version is written for MySQL, which requires:
- Explicit VARCHAR length limits
- InnoDB storage engine for future foreign key support

The table is only created if it does not already exist.
"""

sql_create_hospital = """
CREATE TABLE IF NOT EXISTS Hospital (
    hospital_id INT AUTO_INCREMENT PRIMARY KEY,
    name VARCHAR(150) NOT NULL UNIQUE,
    address VARCHAR(255) NOT NULL,
    size_beds INT NOT NULL CHECK (size_beds > 0),
    hospital_type VARCHAR(100),
    accreditation_status VARCHAR(100)
) ENGINE=InnoDB;
"""

cur.execute(sql_create_hospital)
conn.commit()
print("Hospital table created.")


Hospital table created.


## DOCTOR table

This table contains a list of doctors. Related information should be stored about each doctor including at least:
name, date of birth, Address.

1 hospital can have a relationship with many doctors and the hospital_id field allows this (foreign key) 

| Field Name      | Data Type       | Constraints / Notes                                    |
| --------------- | --------------- | ------------------------------------------------------ |
| `doctor_id`     | INT             | **Primary key**, auto-increment                        |
| `hospital_id`   | INT             | **Foreign key** = Hospital.hospital_id                 |
| `first_name`    | VARCHAR(100)    | Not null                                               |
| `last_name`     | VARCHAR(100)    | Not null                                               |
| `date_of_birth` | DATE            | Not null                                               |
| `specialty`     | VARCHAR(100)    | e.g. Cardiology, Oncology                              |
| `address`       | VARCHAR(255)    |  Not Null                                              |


In [6]:
"""
Creates the `Doctor` table in the MySQL database.

This table stores demographic and professional information for each doctor.
Each doctor is assigned to exactly one hospital, enforced by the foreign key
constraint on `hospital_id`. This version is adapted for MySQL, which enforces
data types strictly and requires VARCHAR lengths and AUTO_INCREMENT syntax.

Fields:
- doctor_id:      Primary key (AUTO_INCREMENT)
- hospital_id:    Foreign key referencing Hospital(hospital_id)
- first_name:     Doctor's given name (required)
- last_name:      Doctor's surname (required)
- date_of_birth:  Stored as a MySQL DATE type (required)
- specialty:      Medical specialty (optional)
- address:        Work or home address (required)

Notes:
- ENGINE=InnoDB is required for foreign key support.
- This table is created only if it does not already exist.
"""

sql_create_doctor = """
CREATE TABLE IF NOT EXISTS Doctor (
    doctor_id INT AUTO_INCREMENT PRIMARY KEY,
    hospital_id INT NOT NULL,
    first_name VARCHAR(100) NOT NULL,
    last_name VARCHAR(100) NOT NULL,
    date_of_birth DATE NOT NULL,
    specialty VARCHAR(100),
    address VARCHAR(255) NOT NULL,
    FOREIGN KEY (hospital_id)
        REFERENCES Hospital(hospital_id)
        ON DELETE CASCADE
        ON UPDATE CASCADE
) ENGINE=InnoDB;
"""

cur.execute(sql_create_doctor)
conn.commit()
print("Doctor table created.")


Doctor table created.


## PATIENT TABLE

Each patient is assigned to one doctor (foreign key)

| Field Name      | Data Type (SQL) | Constraints / Notes                |
| --------------- | --------------- | ---------------------------------- |
| `patient_id`    | INT             | **Primary key**, auto-increment    |
| `doctor_id`     | INT             | **Foreign key** =  Doctor.doctor_id |
| `first_name`    | VARCHAR(100)    | Not null                           |
| `last_name`     | VARCHAR(100)    | Not null                           |
| `date_of_birth` | DATE            | Not null                           |
| `address`       | VARCHAR(255)    | Not null                           |       
| `gender`        | VARCHAR(10)     | Not null                           |


## MEDICATION table

30+ meds. A medication can be prescribed to multiple patients and each patient could be prescribed multiple medications, but that many-to-many relationship will be described in the PRESCRIPTIONS table.


| Field Name.      | Data Type       | Constraints/ Notes               |
| ---------------- | --------------- | -------------------------------- |
| `medication_id`  | INT             | **primary key** autoincrement.   |
| `name`           | VARCHAR(150).   | Unique, not null.                |

## PRESCRIPTTION Table

This is a Junction table for PATIENT and MEDICATION

This table represents the many-to-many relationship between patients and medications. Each record indicates that a specific patient has been prescribed a specific medication.
It also records who prescribed the medication, when it was prescrinbed, and will presumably contain extra information like dose, duration and route. This is called a descriptive junction table, or a bridge entity with attributes.

| Field Name          | Data Type    | Notes                               |
| ------------------- | ------------ | ----------------------------------- |
| `prescription_id`   | INT          | **Primary key**                     |
| `patient_id`        | INT          | **FK = Patient**                    |
| `doctor_id`         | INT          | **FK = Doctor**                     |
| `medication_id`     | INT          | **FK = Medication**                 |
| `prescribed_date`   | DATE         | Must be within past 2 years         |
| `dose_instructions` | VARCHAR(255) | e.g., "Take one tablet twice a day" |
| `duration_days`     | INT          | Optional                            |
| `route`             | VARCHAR(150) | not Null                            |



## DISEASES Table

One disease can be treated by many meds, and by many doctors. 
One medication can treat many diseases.
One doctor can specialise in many diseases.

There are TWO many-to-many relationships here, which can be represented in two junction tables: Disease-Medication, and Disease-Doctor


| Field         | Type         | Notes                  |
| ------------- | ------------ | ---------------------- |
| `disease_id`  | INT          | **Primary key**        |
| `name`        | VARCHAR(150) | Unique, not null       |
| `description` | TEXT         | Optional               |
| `icd10_code`  | VARCHAR(10)  | Optional               |




## DiseaseMEDICATION

Many to many, represents which meds are used to treat which diseases

| Field                  | Type | Notes           |
| ---------------------- | ---- | --------------- |
| `disease_treatment_id` | INT  | Primary Key             |
| `disease_id`           | INT  | Foreign Key - Disease    |
| `medication_id`        | INT  | Foreign Key - Medication |


## DiseaseDOCTOR

Many to Many, represents which doctors treat which diseases 

| Field                   | Type | Notes        |
| ----------------------- | ---- | ------------ |
| `disease_specialist_id` | INT  | PK           |
| `disease_id`            | INT  | FK → Disease |
| `doctor_id`             | INT  | FK → Doctor  |



## APPOINTMENTS table

Each Appointment is a one to one relationship between a PATIENT and a DOCTOR occuring at a HOSPTIAL and at a certain time and for a certain duration.

| Field               | Data Type    | Notes                                      |
| ------------------- | ------------ | ------------------------------------------ |
| `appointment_id`    | INT          | **Primary key**                            |
| `patient_id`        | INT          | **FK → Patient**                           |
| `doctor_id`         | INT          | **FK → Doctor**                            |
| `hospital_id`       | INT          | **FK → Hospital**                          |
| `appointment_start` | DATETIME     | exact scheduled time                       |
| `duration_minutes`  | INT          | typical values 10–60                       |
| `reason`            | VARCHAR(255) | optional free-text                         |
| `status`            | VARCHAR(50)  | e.g. "Scheduled", "Completed", "Cancelled" |


## LABTESTS table

A table should be created for storing the results from lab tests,
which are for individual patients.
A patient may have many lab tests, but each lab result is for one patient.
and each lab test should be requested by a specific doctor (one to one)
each lab result corresponds to one test type (many to one)

To achieve this we can create a LAB TESTStable, and then a LABRESULTS table which is a junctional table with attributes, joining a test result to a patient (many to one) and describing the doctor who requested it, the date of the request, the result of the test, the date of the result, and optionally whether it is normal or abnormal and a reference range.

| Field             | Type         | Notes                                              |
| ----------------- | ------------ | -------------------------------------------------- |
| `lab_test_id`     | INT          | **Primary key**                                    |
| `name`            | VARCHAR(150) | e.g. “HbA1c”, “CRP”                                |
| `description`     | TEXT         | optional                                           |
| `units`           | VARCHAR(20)  | e.g. “mmol/mol”, “mg/L”                            |
| `reference_range` | VARCHAR(50)  |                                                    |
| `sample_type`     | VARCHAR(50)  | e.g. “Blood”, “Urine”                              |


## LABRESULTS Table

| Field            | Type        | Notes                            |
| ---------------- | ----------- | -------------------------------- |
| `lab_result_id`  | INT         | **Primary key**                  |
| `lab_test_id`    | INT         | FK - LabTest                     |
| `patient_id`     | INT         | FK - Patient                     |
| `doctor_id`      | INT         | FK - Doctor                      |
| `requested_date` | DATE        | when the doctor ordered the test |
| `result_date`    | DATE        | when the lab produced the result |
| `result_value`   | VARCHAR(50) | numeric or text result           |
| `is_normal`      | BOOLEAN     | optional                         |
| `notes`          | TEXT        | optional commentary              |
